In [56]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

In [57]:
dftrain=pd.read_csv('AAATrain.csv')

In [58]:
dftest=pd.read_csv('AAATest.csv')

In [59]:
pd.reset_option('all')

In [60]:
model = sm.logit(formula = 'travel_destination_EUR ~  travel_travel_type_Ship + travel_revenue_index+\
travel_tripvalue_index + ers_services_count + travel_travel_month_8 + travel_travel_month_7 + travel_travel_month_6 + travel_travel_type_Insurance +\
travel_provide_name_best_western_inl + travel_channel_BRANCH + travel_channel_EVENT +\
travel_channel_WEB +travel_dom_int_D', data=dftrain).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.037699
         Iterations 11
                             Logit Regression Results                             
Dep. Variable:     travel_destination_EUR   No. Observations:                45800
Model:                              Logit   Df Residuals:                    45786
Method:                               MLE   Df Model:                           13
Date:                    Tue, 06 Aug 2024   Pseudo R-squ.:                  0.4734
Time:                            11:01:02   Log-Likelihood:                -1726.6
converged:                           True   LL-Null:                       -3278.9
Covariance Type:                nonrobust   LLR p-value:                     0.000
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept              

In [61]:
Xtest = dftest[['travel_travel_type_Ship', 'travel_revenue_index', 'travel_tripvalue_index', 'ers_services_count',
     'travel_travel_month_8', 'travel_travel_month_7', 'travel_travel_month_6', 'travel_travel_type_Insurance', 'travel_provide_name_best_western_inl', 
     'travel_channel_BRANCH', 'travel_channel_EVENT', 'travel_channel_WEB', 'travel_dom_int_D']]
ytest = dftest['travel_destination_EUR']

In [62]:
Europe_yhat = model.predict(Xtest)

In [63]:
dftest['Europe_yhat'] = Europe_yhat

In [64]:
print(Europe_yhat)

0        0.000321
1        0.000971
2        0.001040
3        0.000215
4        0.001146
           ...   
45795    0.000415
45796    0.000205
45797    0.007746
45798    0.005717
45799    0.000427
Length: 45800, dtype: float64


In [65]:
dftest = dftest.sort_values(by = 'Europe_yhat', ascending = False)

In [66]:
dftest['Europe_Quintile'] = pd.qcut(dftest['Europe_yhat'], 5, labels=[i for i in range (5, 0, -1)])

In [67]:
print(dftest['Europe_Quintile'])

20722    1
27181    1
5684     1
16380    1
7808     1
        ..
18549    5
40899    5
3015     5
35559    5
2174     5
Name: Europe_Quintile, Length: 45800, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]


In [68]:
dftest['Europe_Quintile'].describe()

count     45800
unique        5
top           3
freq       9386
Name: Europe_Quintile, dtype: int64

In [69]:
dftest['Europe_Quintile'] = dftest['Europe_Quintile'].astype('int')

In [70]:
Europe_travelers = pd.crosstab(dftest['Europe_Quintile'], dftest['travel_destination_EUR'], normalize='index')

In [71]:
print(Europe_travelers)

travel_destination_EUR         0         1
Europe_Quintile                           
1                       0.935917  0.064083
2                       0.999888  0.000112
3                       0.999680  0.000320
4                       1.000000  0.000000
5                       1.000000  0.000000


In [72]:
dftest.to_csv('EURfile.csv')